In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

2022-06-14 15:15:14 Configured OSMnx 1.1.1
2022-06-14 15:15:14 HTTP response caching is on


In [2]:
errors = pd.DataFrame(columns = ['outside'], index=['01','02','03', '04', '05', '06', '07', '08', '09', '10','11', '12','13','14','15','16','17','18', '19','20','21','22','23','24','25','26','27','28','29','30','31','32'])
errors2 = pd.DataFrame(columns = ['outside'], index=['01','02','03', '04', '05', '06', '07', '08', '09', '10','11', '12','13','14','15','16','17','18', '19','20','21','22','23','24','25','26','27','28','29','30','31','32'])

In [3]:
bad_coordinates = gpd.GeoDataFrame()
for i in range(1,33):
    ####Download states
    edo = format(i, "02d")
    print(edo)
    mun_gdf = gpd.GeoDataFrame()
    # Downloads municipality polygon according to code
    query = f"SELECT * FROM marco.mpos_2020 WHERE \"CVE_ENT\" LIKE \'{edo}\'"
    mun_gdf = mun_gdf.append(aup.gdf_from_query(query, geometry_col='geometry'))
    poly_wkt = mun_gdf.dissolve().geometry.to_wkt()[0]
    ### Read schools
    df = pd.read_csv(f'/home/jovyan/work/data/external/sigade/{edo}-concentrado_escuelas.csv', encoding='latin1')
    df.columns= df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "_")
    df.rename(columns = {'ï»¿clave_del_centro_de_trabajo':'clave_del_centro_de_trabajo'}, inplace = True)
    df['clave_de_la_entidad_federativa'] = df['clave_de_la_entidad_federativa'].fillna(0)
    df['clave_del_municipio_o_delegaciã³n'] = df['clave_del_municipio_o_delegaciã³n'].fillna(0)
    df = df.astype({"clave_de_la_entidad_federativa": int}, errors='raise') 
    df = df.astype({"clave_del_municipio_o_delegaciã³n": int}, errors='raise') 
    for column in df.columns[df.dtypes=='object']:
        df[column] = df[column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    df['x'] = df['ubicaciã³n_de_la_escuela-localidad_al_oeste_del_meridiano_de_greenwich,_expresada_en_grados']
    df['y'] = df['ubicaciã³n_de_la_escuela-localidad_al_norte_del_ecuador,_expresada_en_grados']
    for idx,row in df.iterrows():
        if df.loc[idx, 'x']>=0:
            df.loc[idx, 'x'] = df.loc[idx, 'x']*-1
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x, df.y), crs="EPSG:4326" )
    gdf["CVEGEOMUN"] = gdf.apply(
        lambda row: "{:02d}".format((row.loc["clave_de_la_entidad_federativa"]))
        + "{:03d}".format((row.loc["clave_del_municipio_o_delegaciã³n"])),
        axis=1,
        )
    print('code done')
    gdf = gdf.drop_duplicates(subset=['clave_del_centro_de_trabajo'], keep = 'first')
    gdf = gdf.set_index('clave_del_centro_de_trabajo')
    #####Check if within or outside
    mun_poly = mun_gdf.unary_union
    within_mun = gdf[gdf.geometry.within(mun_poly)]
    outside_mun = gdf[~gdf.geometry.within(mun_poly)]
    drop = within_mun.drop_duplicates(subset=['geometry'], keep = False)
    drop_clean = drop[['geometry']]
    within_clean = within_mun[['geometry']]
    checker = within_clean.merge(drop_clean, left_index=True, right_index=True, how='outer', indicator=True)
    g2g = checker[checker['_merge'] == 'both']
    duplicates = checker[checker['_merge'] == 'left_only']
    duplicates = duplicates[['geometry_x']]
    find = duplicates.merge(gdf, left_index=True, right_index=True, how='outer', indicator=True)
    find = find[find['_merge'] == 'both']
    find.drop(['_merge', 'geometry_x'], axis=1, inplace = True)
    find = find.append(outside_mun)
    bad_coordinates = bad_coordinates.append(find)
    errors.loc[edo, 'outside'] = len(find)

01
code done
02
code done
03
code done
04
code done
05
code done
06
code done
07
code done
08
code done
09
code done
10
code done
11
code done
12
code done
13
code done
14
code done
15
code done
16
code done
17
code done
18
code done
19
code done
20
code done
21
code done
22
code done
23
code done
24
code done
25
code done
26
code done
27
code done
28
code done
29
code done
30
code done
31
code done
32
code done


In [4]:
errors['outside']

01     1432
02     3899
03      585
04     1736
05     4468
06     1553
07    16602
08     6274
09     9395
10      354
11     4329
12     6034
13     2753
14     7822
15    10766
16    10128
17     1859
18     3596
19     4514
20    13673
21     8973
22     1110
23     1499
24     8299
25     2648
26     2653
27     4518
28     4119
29      774
30     6662
31     1870
32     5036
Name: outside, dtype: object

In [5]:
### Exclude those not within the 74 metropolitan areas
cities = pd.read_json('/home/jovyan/work/scripts/Metropolis_CVE.json')
find_poi = gpd.GeoDataFrame()
for c in cities.columns.unique():
    # Iterates over municipality codes for each metropolitan area or capital
    for j in range(len(cities.loc["mpos", c])):
        # Extracts specific municipality code
        m = cities.loc["mpos", c][j]
        sep_in_mun = bad_coordinates[bad_coordinates['CVEGEOMUN']==m]
        find_poi = find_poi.append(sep_in_mun)
    errors2.loc[edo, 'outside'] = len(find_poi)

In [6]:
errors2

,outside
01,NaN
02,NaN
03,NaN
04,NaN
05,NaN
06,NaN
07,NaN
08,NaN
09,NaN
10,NaN
